In [2]:
!pip3 list

Package                      Version
---------------------------- -----------
absl-py                      1.3.0
APScheduler                  3.6.3
asttokens                    2.1.0
astunparse                   1.6.3
backcall                     0.2.0
beautifulsoup4               4.11.1
bitarray                     2.6.0
bs4                          0.0.1
cachetools                   4.2.2
certifi                      2021.10.8
charset-normalizer           2.0.12
click                        8.1.3
colorama                     0.4.4
contourpy                    1.0.6
cycler                       0.11.0
debugpy                      1.6.3
decorator                    5.1.1
entrypoints                  0.4
executing                    1.2.0
flatbuffers                  22.10.26
fonttools                    4.38.0
gast                         0.4.0
google-auth                  2.14.1
google-auth-oauthlib         0.4.6
google-pasta                 0.2.0
grpcio                       1.50.0
h